# Interactive Fantasy Football Draft Board

Real-time draft tracking with player removal, team rosters, and value-based recommendations.

## Features:
- ✅ Live player availability tracking
- ✅ 14-team roster management
- ✅ Value-based drafting recommendations
- ✅ Position scarcity alerts
- ✅ Draft state persistence
- ✅ Multiple interactive views

In [1]:
# Required imports
import pandas as pd
import numpy as np
import yaml
import json
import pickle
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

print("📊 Fantasy Football Interactive Draft Board")
print("🚀 Loading data and initializing draft system...")

📊 Fantasy Football Interactive Draft Board
🚀 Loading data and initializing draft system...


In [2]:
# Load configuration and data
with open('config/league-config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Load draft cheat sheet (our main player rankings)
cheat_sheet = pd.read_csv('draft_cheat_sheet.csv')

# Load position-specific projections for additional context
projections = {}
positions = ['qb', 'rb', 'wr', 'te']
for pos in positions:
    try:
        df = pd.read_csv(f'data/projections/projections_{pos}_20250814.csv')
        projections[pos.upper()] = df
    except FileNotFoundError:
        print(f"⚠️ Warning: {pos} projections not found")

print(f"✅ Loaded {len(cheat_sheet)} players")
print(f"✅ League: {config['league_name']} ({config['basic_settings']['teams']} teams)")
print(f"✅ Draft: {config['draft']['type']} draft")

✅ Loaded 50 players
✅ League: Fantasy 24-25 (14 teams)
✅ Draft: Snake draft


In [3]:
class DraftTracker:
    """Interactive draft tracking system"""
    
    def __init__(self, config, cheat_sheet):
        self.config = config
        self.num_teams = config['basic_settings']['teams']
        self.roster_size = config['roster']['total_size']
        self.team_names = config['team_names']
        
        # Initialize player data
        self.available_players = cheat_sheet.copy()
        self.available_players['Available'] = True
        self.available_players['Drafted_By'] = None
        self.available_players['Pick_Number'] = None
        self.available_players['Round'] = None
        
        # Draft state
        self.current_pick = 1
        self.current_round = 1
        self.draft_history = []
        
        # Team rosters (dict of team_name -> list of players)
        self.team_rosters = {team: [] for team in self.team_names}
        
        # Snake draft order
        self.draft_order = self._generate_snake_order()
        
    def _generate_snake_order(self):
        """Generate snake draft pick order"""
        order = []
        for round_num in range(1, self.roster_size + 1):
            if round_num % 2 == 1:  # Odd rounds: 1, 2, 3, ..., 14
                round_picks = [(pick, team) for pick, team in enumerate(self.team_names, 1)]
            else:  # Even rounds: 14, 13, 12, ..., 1
                round_picks = [(pick, team) for pick, team in enumerate(reversed(self.team_names), 1)]
            
            for team_pos, team in round_picks:
                overall_pick = (round_num - 1) * self.num_teams + team_pos
                order.append({
                    'overall_pick': overall_pick,
                    'round': round_num,
                    'pick_in_round': team_pos,
                    'team': team
                })
        return order
    
    def get_current_team(self):
        """Get team currently on the clock"""
        if self.current_pick <= len(self.draft_order):
            return self.draft_order[self.current_pick - 1]['team']
        return None
    
    def draft_player(self, player_name, team_override=None):
        """Draft a player to current team (or override team)"""
        if self.current_pick > len(self.draft_order):
            print("🏁 Draft is complete!")
            return False
            
        # Find player
        player_idx = self.available_players[self.available_players['Player'] == player_name].index
        if len(player_idx) == 0:
            print(f"❌ Player '{player_name}' not found or already drafted")
            return False
            
        player_idx = player_idx[0]
        
        # Get team and pick info
        current_team = team_override or self.get_current_team()
        pick_info = self.draft_order[self.current_pick - 1]
        
        # Update player data
        self.available_players.loc[player_idx, 'Available'] = False
        self.available_players.loc[player_idx, 'Drafted_By'] = current_team
        self.available_players.loc[player_idx, 'Pick_Number'] = self.current_pick
        self.available_players.loc[player_idx, 'Round'] = pick_info['round']
        
        # Add to team roster
        player_data = self.available_players.loc[player_idx].to_dict()
        self.team_rosters[current_team].append(player_data)
        
        # Add to draft history
        self.draft_history.append({
            'pick': self.current_pick,
            'round': pick_info['round'],
            'pick_in_round': pick_info['pick_in_round'],
            'team': current_team,
            'player': player_name,
            'position': player_data['Position'],
            'vbd': player_data['Custom_VBD'],
            'timestamp': datetime.now().strftime('%H:%M:%S')
        })
        
        print(f"✅ Pick {self.current_pick}: {current_team} selects {player_name} ({player_data['Position']})")
        
        # Advance to next pick
        self.current_pick += 1
        if self.current_pick > self.num_teams:
            if (self.current_pick - 1) % self.num_teams == 0:
                self.current_round += 1
        
        return True
    
    def undo_last_pick(self):
        """Undo the last draft pick"""
        if not self.draft_history:
            print("❌ No picks to undo")
            return False
            
        last_pick = self.draft_history.pop()
        player_name = last_pick['player']
        team = last_pick['team']
        
        # Find and restore player
        player_idx = self.available_players[self.available_players['Player'] == player_name].index[0]
        self.available_players.loc[player_idx, 'Available'] = True
        self.available_players.loc[player_idx, 'Drafted_By'] = None
        self.available_players.loc[player_idx, 'Pick_Number'] = None
        self.available_players.loc[player_idx, 'Round'] = None
        
        # Remove from team roster
        self.team_rosters[team] = [p for p in self.team_rosters[team] if p['Player'] != player_name]
        
        # Revert pick counter
        self.current_pick -= 1
        self.current_round = ((self.current_pick - 1) // self.num_teams) + 1
        
        print(f"↩️ Undid pick: {player_name} returned to available players")
        return True
    
    def get_available_by_position(self, position=None, top_n=20):
        """Get available players, optionally filtered by position"""
        available = self.available_players[self.available_players['Available'] == True].copy()
        
        if position and position != 'ALL':
            available = available[available['Position'] == position]
        
        return available.head(top_n)[['Draft_Rank', 'Player', 'Position', 'Team', 'Bye', 'ECR', 'Custom_VBD']]
    
    def get_team_needs(self, team_name):
        """Analyze team positional needs"""
        roster = self.team_rosters[team_name]
        position_counts = {}
        for player in roster:
            pos = player['Position']
            position_counts[pos] = position_counts.get(pos, 0) + 1
        
        # Required starters from config
        required = self.config['roster']['roster_slots']
        maximums = self.config['roster']['maximums']
        
        needs = {}
        for pos, req in required.items():
            if pos == 'FLEX':
                continue  # Handle flex separately
            current = position_counts.get(pos, 0)
            max_allowed = maximums.get(pos, 999)
            needs[pos] = {
                'required': req,
                'current': current,
                'needed': max(0, req - current),
                'can_draft': max_allowed - current
            }
        
        return needs
    
    def save_draft_state(self, filename='draft_state.pkl'):
        """Save current draft state"""
        state = {
            'available_players': self.available_players,
            'team_rosters': self.team_rosters,
            'draft_history': self.draft_history,
            'current_pick': self.current_pick,
            'current_round': self.current_round
        }
        with open(filename, 'wb') as f:
            pickle.dump(state, f)
        print(f"💾 Draft state saved to {filename}")
    
    def load_draft_state(self, filename='draft_state.pkl'):
        """Load draft state from file"""
        try:
            with open(filename, 'rb') as f:
                state = pickle.load(f)
            
            self.available_players = state['available_players']
            self.team_rosters = state['team_rosters']
            self.draft_history = state['draft_history']
            self.current_pick = state['current_pick']
            self.current_round = state['current_round']
            print(f"📂 Draft state loaded from {filename}")
            return True
        except FileNotFoundError:
            print(f"❌ File {filename} not found")
            return False

# Initialize draft tracker
draft = DraftTracker(config, cheat_sheet)
print(f"\n🏈 Draft initialized! Team on the clock: {draft.get_current_team()}")


🏈 Draft initialized! Team on the clock: Bam Bam Kam


In [4]:
# Interactive Draft Controls
class DraftInterface:
    def __init__(self, draft_tracker):
        self.draft = draft_tracker
        self.setup_widgets()
        
    def setup_widgets(self):
        # Draft controls
        self.player_search = widgets.Text(
            placeholder='Search player name...',
            description='Search:',
            style={'description_width': 'initial'}
        )
        
        self.position_filter = widgets.Dropdown(
            options=['ALL', 'QB', 'RB', 'WR', 'TE', 'DEF', 'K'],
            value='ALL',
            description='Position:'
        )
        
        self.player_dropdown = widgets.Dropdown(
            options=[],
            description='Select Player:',
            style={'description_width': 'initial'}
        )
        
        self.team_override = widgets.Dropdown(
            options=['CURRENT'] + self.draft.team_names,
            value='CURRENT',
            description='Draft To:',
            style={'description_width': 'initial'}
        )
        
        self.draft_button = widgets.Button(
            description='DRAFT PLAYER',
            button_style='success',
            icon='check'
        )
        
        self.undo_button = widgets.Button(
            description='UNDO LAST',
            button_style='warning',
            icon='undo'
        )
        
        self.save_button = widgets.Button(
            description='SAVE DRAFT',
            button_style='info',
            icon='save'
        )
        
        self.load_button = widgets.Button(
            description='LOAD DRAFT',
            button_style='info',
            icon='upload'
        )
        
        # Output areas
        self.status_output = widgets.Output()
        self.players_output = widgets.Output()
        self.rosters_output = widgets.Output()
        self.analytics_output = widgets.Output()
        
        # Event handlers
        self.player_search.observe(self.update_player_list, names='value')
        self.position_filter.observe(self.update_player_list, names='value')
        self.draft_button.on_click(self.draft_player)
        self.undo_button.on_click(self.undo_pick)
        self.save_button.on_click(self.save_draft)
        self.load_button.on_click(self.load_draft)
        
        # Initial update
        self.update_player_list()
        self.refresh_all_displays()
        
    def update_player_list(self, change=None):
        """Update available players dropdown based on search/filter"""
        available = self.draft.available_players[self.draft.available_players['Available'] == True].copy()
        
        # Apply position filter
        if self.position_filter.value != 'ALL':
            available = available[available['Position'] == self.position_filter.value]
        
        # Apply search filter
        if self.player_search.value:
            search_term = self.player_search.value.lower()
            available = available[available['Player'].str.lower().str.contains(search_term, na=False)]
        
        # Update dropdown
        available = available.sort_values('Draft_Rank').head(50)
        options = [(f"{row['Draft_Rank']}. {row['Player']} ({row['Position']}, {row['Team']})", row['Player']) 
                  for _, row in available.iterrows()]
        
        self.player_dropdown.options = options
        if options:
            self.player_dropdown.value = options[0][1]
        
        self.refresh_players_display()
        
    def draft_player(self, button):
        """Handle draft button click"""
        if not self.player_dropdown.value:
            return
            
        team = None if self.team_override.value == 'CURRENT' else self.team_override.value
        
        with self.status_output:
            clear_output()
            success = self.draft.draft_player(self.player_dropdown.value, team)
            
        if success:
            self.update_player_list()
            self.refresh_all_displays()
            
    def undo_pick(self, button):
        """Handle undo button click"""
        with self.status_output:
            clear_output()
            success = self.draft.undo_last_pick()
            
        if success:
            self.update_player_list()
            self.refresh_all_displays()
            
    def save_draft(self, button):
        """Save draft state"""
        with self.status_output:
            clear_output()
            self.draft.save_draft_state()
            
    def load_draft(self, button):
        """Load draft state"""
        with self.status_output:
            clear_output()
            success = self.draft.load_draft_state()
            
        if success:
            self.update_player_list()
            self.refresh_all_displays()
            
    def refresh_players_display(self):
        """Refresh available players table"""
        with self.players_output:
            clear_output()
            
            # Current pick info
            current_team = self.draft.get_current_team()
            print(f"🎯 Pick {self.draft.current_pick} (Round {self.draft.current_round})")
            print(f"⏰ ON THE CLOCK: {current_team}")
            print("\n" + "="*60)
            
            # Available players
            available = self.draft.get_available_by_position(self.position_filter.value, 15)
            if not available.empty:
                display(HTML(available.to_html(index=False, classes='table table-striped')))
            else:
                print("No available players match current filters")
                
    def refresh_rosters_display(self):
        """Refresh team rosters display"""
        with self.rosters_output:
            clear_output()
            
            print("📋 TEAM ROSTERS\n")
            
            for i, team in enumerate(self.draft.team_names):
                if i % 2 == 0:  # Start new row every 2 teams
                    print("\n" + "="*120)
                
                roster = self.draft.team_rosters[team]
                needs = self.draft.get_team_needs(team)
                
                print(f"\n{team} ({len(roster)}/{self.draft.roster_size})")
                print("-" * 40)
                
                if roster:
                    for player in roster:
                        pick_info = f"R{player['Round']}.{player['Pick_Number'] % self.draft.num_teams or self.draft.num_teams}"
                        print(f"{pick_info:>6} {player['Player']:<20} {player['Position']:<3} {player['Custom_VBD']:>6.1f}")
                else:
                    print("No picks yet")
                    
                # Show positional needs
                need_summary = []
                for pos, need in needs.items():
                    if need['needed'] > 0:
                        need_summary.append(f"{pos}({need['needed']})")
                
                if need_summary:
                    print(f"Needs: {', '.join(need_summary)}")
                    
    def refresh_analytics_display(self):
        """Refresh draft analytics"""
        with self.analytics_output:
            clear_output()
            
            if not self.draft.draft_history:
                print("📊 Draft analytics will appear after picks are made")
                return
                
            # Recent picks
            print("📈 RECENT PICKS\n")
            recent = self.draft.draft_history[-10:]  # Last 10 picks
            for pick in reversed(recent):
                print(f"Pick {pick['pick']:3d}: {pick['team']:<20} {pick['player']:<25} {pick['position']} ({pick['vbd']:6.1f})")
            
            # Position analysis
            print("\n🎯 POSITION ANALYSIS\n")
            pos_counts = {}
            for pick in self.draft.draft_history:
                pos = pick['position']
                pos_counts[pos] = pos_counts.get(pos, 0) + 1
            
            for pos, count in sorted(pos_counts.items()):
                remaining = len(self.draft.available_players[
                    (self.draft.available_players['Position'] == pos) & 
                    (self.draft.available_players['Available'] == True)
                ])
                print(f"{pos:>3}: {count:2d} drafted, {remaining:3d} available")
                
    def refresh_all_displays(self):
        """Refresh all display areas"""
        self.refresh_players_display()
        self.refresh_rosters_display()
        self.refresh_analytics_display()
        
    def display_interface(self):
        """Display the complete interface"""
        # Control panel
        controls = widgets.VBox([
            widgets.HTML("<h3>🏈 Draft Controls</h3>"),
            widgets.HBox([self.player_search, self.position_filter]),
            self.player_dropdown,
            widgets.HBox([self.team_override, self.draft_button]),
            widgets.HBox([self.undo_button, self.save_button, self.load_button]),
            self.status_output
        ])
        
        # Tabbed display
        tab = widgets.Tab(children=[
            self.players_output,
            self.rosters_output, 
            self.analytics_output
        ])
        tab.set_title(0, '📊 Available Players')
        tab.set_title(1, '📋 Team Rosters')
        tab.set_title(2, '📈 Draft Analytics')
        
        return widgets.VBox([controls, tab])

# Create and display interface
interface = DraftInterface(draft)
display(interface.display_interface())

In [5]:
# Draft Board Visualization
def create_draft_board_viz():
    """Create visual draft board"""
    if not draft.draft_history:
        print("📊 Draft board will appear after picks are made")
        return
    
    # Prepare data for visualization
    board_data = []
    for pick in draft.draft_history:
        board_data.append({
            'Pick': pick['pick'],
            'Round': pick['round'],
            'Team': pick['team'],
            'Player': pick['player'],
            'Position': pick['position'],
            'VBD': pick['vbd']
        })
    
    df_board = pd.DataFrame(board_data)
    
    # Create color mapping for positions
    pos_colors = {
        'QB': '#FF6B6B',   # Red
        'RB': '#4ECDC4',   # Teal  
        'WR': '#45B7D1',   # Blue
        'TE': '#96CEB4',   # Green
        'DEF': '#FFEAA7',  # Yellow
        'K': '#DDA0DD'     # Plum
    }
    
    # Draft board heatmap
    fig = go.Figure()
    
    for _, row in df_board.iterrows():
        round_num = row['Round']
        pick_in_round = ((row['Pick'] - 1) % draft.num_teams) + 1
        
        fig.add_trace(go.Scatter(
            x=[pick_in_round],
            y=[round_num],
            mode='markers+text',
            marker=dict(
                size=30,
                color=pos_colors.get(row['Position'], '#95A5A6'),
                line=dict(width=2, color='white')
            ),
            text=row['Position'],
            textposition="middle center",
            textfont=dict(size=10, color='white'),
            hovertemplate=(
                f"<b>Pick {row['Pick']}</b><br>"
                f"Round {row['Round']}, Pick {pick_in_round}<br>"
                f"<b>{row['Player']}</b><br>"
                f"{row['Position']} - {row['Team']}<br>"
                f"VBD: {row['VBD']:.1f}<extra></extra>"
            ),
            showlegend=False
        ))
    
    # Customize layout
    fig.update_layout(
        title=f"🏈 Draft Board - Through Pick {len(draft.draft_history)}",
        xaxis=dict(
            title="Pick in Round",
            tickmode='linear',
            tick0=1,
            dtick=1,
            range=[0.5, draft.num_teams + 0.5]
        ),
        yaxis=dict(
            title="Round",
            tickmode='linear', 
            tick0=1,
            dtick=1,
            autorange='reversed'
        ),
        height=600,
        plot_bgcolor='white',
        font=dict(size=12)
    )
    
    fig.show()
    
    # Position distribution pie chart
    pos_counts = df_board['Position'].value_counts()
    
    fig2 = go.Figure(data=[go.Pie(
        labels=pos_counts.index,
        values=pos_counts.values,
        marker_colors=[pos_colors.get(pos, '#95A5A6') for pos in pos_counts.index],
        textinfo='label+percent',
        title="Position Distribution"
    )])
    
    fig2.update_layout(
        title="📊 Positions Drafted So Far",
        height=400
    )
    
    fig2.show()

# Button to refresh visualizations
viz_button = widgets.Button(
    description='📊 Update Draft Board',
    button_style='primary',
    icon='chart-bar'
)

def update_viz(button):
    create_draft_board_viz()

viz_button.on_click(update_viz)
display(viz_button)

Button(button_style='primary', description='📊 Update Draft Board', icon='chart-bar', style=ButtonStyle())

In [6]:
# Quick Draft Actions for Testing
def quick_draft_top_players(num_picks=5):
    """Quick draft top available players for testing"""
    print(f"🚀 Quick drafting top {num_picks} players...\n")
    
    for i in range(num_picks):
        if draft.current_pick > len(draft.draft_order):
            break
            
        # Get top available player
        available = draft.get_available_by_position(top_n=1)
        if available.empty:
            break
            
        top_player = available.iloc[0]['Player']
        draft.draft_player(top_player)
    
    # Refresh interface
    interface.update_player_list()
    interface.refresh_all_displays()
    print("\n✅ Quick draft complete!")

# Quick action buttons
quick_5 = widgets.Button(description='Quick Draft 5', button_style='warning')
quick_14 = widgets.Button(description='Quick Draft Round 1', button_style='warning')
reset_draft = widgets.Button(description='🔄 Reset Draft', button_style='danger')

def quick_5_handler(button):
    quick_draft_top_players(5)
    
def quick_14_handler(button):
    quick_draft_top_players(14)
    
def reset_handler(button):
    global draft, interface
    draft = DraftTracker(config, cheat_sheet)
    interface = DraftInterface(draft)
    print("🔄 Draft reset to beginning")
    
quick_5.on_click(quick_5_handler)
quick_14.on_click(quick_14_handler)
reset_draft.on_click(reset_handler)

print("⚡ Quick Actions (for testing):")
display(widgets.HBox([quick_5, quick_14, reset_draft]))

⚡ Quick Actions (for testing):


## 🎯 How to Use This Draft Board

### Basic Drafting:
1. **Current Pick**: Shows which team is on the clock
2. **Search/Filter**: Find players by name or position
3. **Select Player**: Choose from dropdown of available players
4. **Draft**: Click "DRAFT PLAYER" to make the pick

### Advanced Features:
- **Team Override**: Draft to a different team than current pick
- **Undo**: Reverse the last pick if needed
- **Save/Load**: Persist draft state between sessions
- **Multiple Tabs**: View available players, team rosters, and analytics

### Draft Visualizations:
- Click "📊 Update Draft Board" for visual draft tracking
- Position-colored draft board shows picks by round
- Pie chart shows position distribution

### Quick Testing:
- Use "Quick Draft" buttons to simulate picks
- "Reset Draft" to start over

### Pro Tips:
- **Monitor Needs**: Check team rosters tab for positional requirements
- **Value Tracking**: Custom VBD scores help identify value picks
- **Scarcity Watch**: Analytics tab shows remaining players by position
- **Save Often**: Use save/load to preserve your draft progress

---

**Ready to draft? Your league awaits! 🏆**